In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define batch size, image size, and input shape for the model
batch_size = 32
image_size = (224, 224)
input_shape = image_size + (3,) # 3 channels for RGB
keras = tf.keras 

# Create an ImageDataGenerator for data augmentation
img_data_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # Preprocess input as per ResNet50
    rotation_range=20,  # Rotate images by up to 20 degrees
    horizontal_flip=True  # Flip images horizontally
)

# Load and prepare training dataset
train_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Train',  # Training data directory
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Classes are encoded as a binary matrix
    shuffle=True,  # Shuffle the data
    seed=123  # Seed for reproducibility
)

# Extract class names from the training dataset
classes = list(train_ds.class_indices.keys())

# Load and prepare validation and test datasets in a similar way
val_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Valid',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=123
)

test_ds = img_data_gen.flow_from_directory(
    '../PotatoImagesData/Test',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    seed=123
)
train_ds.class_indices

Found 900 images belonging to 3 classes.
Found 300 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [2]:
# Initialize ResNet152V2 as the base model with pre-trained ImageNet weights
base_model = ResNet152V2(
    weights="imagenet",
    include_top=False,  # Exclude the top layer to allow for custom layers
    input_shape=input_shape
)

# Freeze the layers of the base model to prevent them from being updated during training
base_model.trainable = False

In [3]:
# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling
x = Dense(1024, activation='relu')(x)  # Dense layer with 1024 neurons
predictions = Dense(train_ds.num_classes, activation='softmax')(x)  # Output layer with softmax activation

In [6]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with an optimizer, loss function, and metrics
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

# Set up early stopping to monitor validation loss and stop training if it doesn't improve
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Train the model using the training and validation datasets
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping]
)

# Evaluate the model on the test dataset
evaluation_results = model.evaluate(test_ds)
print(f"Test Loss: {evaluation_results[0]}, Test Accuracy: {evaluation_results[1]}")

# Save the trained model
model.save('resnet152v2.keras')

Epoch 1/50
29/29 [==============================] - 227s 8s/step - loss: 32.1755 - accuracy: 0.6322 - val_loss: 10.6421 - val_accuracy: 0.7667
Epoch 2/50
29/29 [==============================] - 221s 8s/step - loss: 11.0933 - accuracy: 0.7544 - val_loss: 8.2384 - val_accuracy: 0.7567
Epoch 3/50
29/29 [==============================] - 222s 8s/step - loss: 7.4958 - accuracy: 0.7722 - val_loss: 13.0858 - val_accuracy: 0.7067
Epoch 4/50
29/29 [==============================] - 222s 8s/step - loss: 6.2569 - accuracy: 0.8022 - val_loss: 13.7150 - val_accuracy: 0.6433
Epoch 5/50
29/29 [==============================] - 222s 8s/step - loss: 14.1076 - accuracy: 0.7078 - val_loss: 13.3773 - val_accuracy: 0.7200
Epoch 6/50
29/29 [==============================] - 224s 8s/step - loss: 9.1789 - accuracy: 0.7867 - val_loss: 6.2048 - val_accuracy: 0.8133
Epoch 7/50
29/29 [==============================] - 226s 8s/step - loss: 7.3363 - accuracy: 0.8089 - val_loss: 7.9952 - val_accuracy: 0.7567
Epoch 

In [4]:
model = tf.keras.models.load_model("resnet152v2.keras")

In [8]:
import tensorflow as tf
import numpy as np

# Function to load and preprocess an image
def get_img_array(image_path, target_size=image_size):
    # Load the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # Preprocess the image as per ResNet50
    img = preprocess_input(img)

    return img

# Make predictions on new images
predictions = model.predict(np.array([
    get_img_array("../PotatoImagesData/Train/Potato___Early_blight/bb07a1b7-b9ad-4016-a013-9ff0e4636d4a___RS_Early.B 7156.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___Late_blight/fd35fb86-6558-40f2-b8ac-6922e2107af5___RS_LB 4095.JPG"),
    get_img_array("../PotatoImagesData/Train/Potato___healthy/Potato_healthy-103-_0_3577.jpg"),
]))

# Print out the predicted classes for each image
for i in predictions:
    print(classes[np.argmax(i)])

1/1 [==============================] - 0s 355ms/step
Potato___Early_blight
Potato___healthy
Potato___healthy
